In [ ]:
#https://www.kaggle.com/huanntran100/spotify-song-popularity-prediction/data

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
from scipy import stats
import plotly
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import plotnine
from plotnine import *
import itertools
import math
from prettytable import PrettyTable
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection, preprocessing
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import plot_roc_curve, balanced_accuracy_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
import xgboost as xgb
from scipy.spatial import distance
from imblearn.over_sampling import SMOTE
import copy
import warnings
warnings.filterwarnings("ignore")
plotly.offline.init_notebook_mode(connected = True)
%matplotlib inline
sns.set(style = "ticks", context = "talk")

In [ ]:
# Constants

# categories map
CATEGORY_DICT = {0: "All", 1: "Acoustic", 2: "Chill", 3: "Dance", 4: "Happy", 5: "Loud", 6: "A Capella", 7: "Alternative", 8: "Blues", 9: "Classical", 10: "Country", 11: "Dance", 12: "Electronic", 13: "Folk", 14: "Hip-Hop", 15: "Indie", 16: "Jazz", 17: "Movie", 18: "Opera", 19: "Pop", 20: "R&B", 21: "Rap", 22: "Reggae", 23: "Reggaeton", 24: "Rock", 25: "Ska", 26: "Soul", 27: "Soundtrack", 28: "World"}

# All dataset paths
path_all = "/content/SpotifyFeatures_Cleaned.csv" 
path_acoustic = "/content/SpotifyFeatures_Cleaned_Acoustic.csv"
path_chill = "/content/SpotifyFeatures_Cleaned_Chill.csv"
path_dance = "/content/SpotifyFeatures_Cleaned_Dance.csv"
path_happy = "/content/SpotifyFeatures_Cleaned_Happy.csv"
path_loud = "/content/SpotifyFeatures_Cleaned_Loud.csv"

# Methods
def get_data(csv_path:str)->pd.DataFrame:
  """
    Return a pd.DataFrame for the csv at csv_path
  """
  df = pd.read_csv(csv_path)
  if "Unnamed: 0" in df.columns:
    df = df.drop(["Unnamed: 0"], axis = 1)
  if "cluster" in df.columns:
    df = df.drop(["cluster"], axis = 1)
  return df

def select_data(category:int)->pd.DataFrame:
  """
    Return the data set corresponds to the category number in the CATEGORY_DICT
  """
  if category == 1:
    return get_data(path_acoustic)
  elif category == 2:
    return get_data(path_chill)
  elif category == 3:
    return get_data(path_dance)
  elif category == 4:
    return get_data(path_happy)
  elif category == 5:
    return get_data(path_loud)
  elif category > 5 and category < 29:
    genre = CATEGORY_DICT[category]
    data_all = get_data(path_all)
    return data_all[data_all["genre"] == genre]
  else:
    return get_data(path_all)

In [ ]:
# Select the correct data set
data = select_data(0)
data

,track_id,popularity,acousticness,danceability,liveness,loudness,speechiness,tempo,valence,genre,artist_name,track_name
0,000CzNKC8PEt1yC3L8dqwV,5.0,0.2490,0.518,0.3330,-6.248,0.0407,79.124,0.841,Movie,Henri Salvador,Coeur Brisé à Prendre - Remastered
1,000DfZJww8KiixTKuk9usJ,30.0,0.3660,0.631,0.1090,-6.376,0.0293,120.365,0.307,Reggae,Mike Love,Earthlings
2,000EWWBkYaREzsBplYjUag,39.0,0.8150,0.768,0.1130,-13.284,0.0747,76.430,0.560,Jazz,Don Philippe,Fewerdolr
3,000xQL6tZNLJzIrtIgxqSl,70.0,0.1310,0.748,0.0852,-6.029,0.0644,120.963,0.524,Dance,ZAYN,Still Got Time
4,001KkOBeRiQ1J7IEJYHODW,31.0,0.0697,0.279,0.0974,-9.414,0.0292,130.627,0.110,World,C3 Music,The Father's Love - Live
...,...,...,...,...,...,...,...,...,...,...,...,...
142678,7zz3cHALU9cj7Io5qINt1R,21.0,0.8330,0.353,0.1100,-8.232,0.0267,172.842,0.287,World,Hillsong Worship,Emmanuel
142679,7zz7MbCb9G7KJc1NVl9bL0,38.0,0.4000,0.851,0.1230,-4.641,0.0309,111.993,0.889,Jazz,Caravan Palace,Maniac
142680,7zzFNNxVD0h0ctAT08H0pa,40.0,0.9730,0.538,0.3590,-17.711,0.0569,135.640,0.110,Jazz,Thelonious Monk Quartet,Crepescule With Nellie - Live At Carnegie Hall...
142681,7zzTeItz93lYI52hlcipm5,28.0,0.1130,0.716,0.1510,-5.913,0.1350,136.069,0.544,Reggaeton,Magnate Y Valentino,Una En Un Millón


### Normalize Numeric features

Use min max because most numerical features has an upper and lower bound

In [ ]:
# Min max scale data
scaler = MinMaxScaler()
num_data = data.select_dtypes(exclude=['object'])
num_data = num_data.drop(['popularity'], axis=1)
scaler = scaler.fit(num_data)
data = data.drop(num_data.columns, axis=1)
data[num_data.columns] = scaler.transform(num_data)
data

,track_id,popularity,genre,artist_name,track_name,acousticness,danceability,liveness,loudness,speechiness,tempo,valence
0,000CzNKC8PEt1yC3L8dqwV,5.0,Movie,Henri Salvador,Coeur Brisé à Prendre - Remastered,0.249999,0.495753,0.326487,0.855057,0.019622,0.232707,0.841
1,000DfZJww8KiixTKuk9usJ,30.0,Reggae,Mike Love,Earthlings,0.367469,0.617245,0.100300,0.852689,0.007531,0.429591,0.307
2,000EWWBkYaREzsBplYjUag,39.0,Jazz,Don Philippe,Fewerdolr,0.818273,0.764541,0.104339,0.724862,0.055685,0.219846,0.560
3,000xQL6tZNLJzIrtIgxqSl,70.0,Dance,ZAYN,Still Got Time,0.131525,0.743038,0.076268,0.859110,0.044760,0.432446,0.524
4,001KkOBeRiQ1J7IEJYHODW,31.0,World,C3 Music,The Father's Love - Live,0.069979,0.238792,0.088587,0.796473,0.007425,0.478582,0.110
...,...,...,...,...,...,...,...,...,...,...,...,...
142678,7zz3cHALU9cj7Io5qINt1R,21.0,World,Hillsong Worship,Emmanuel,0.836345,0.318353,0.101310,0.818345,0.004773,0.680115,0.287
142679,7zz7MbCb9G7KJc1NVl9bL0,38.0,Jazz,Caravan Palace,Maniac,0.401606,0.853779,0.114437,0.884793,0.009228,0.389623,0.889
142680,7zzFNNxVD0h0ctAT08H0pa,40.0,Jazz,Thelonious Monk Quartet,Crepescule With Nellie - Live At Carnegie Hall...,0.976908,0.517256,0.352741,0.642944,0.036805,0.502513,0.110
142681,7zzTeItz93lYI52hlcipm5,28.0,Reggaeton,Magnate Y Valentino,Una En Un Millón,0.113453,0.708633,0.142710,0.861256,0.119644,0.504562,0.544


# Modeling

### Train test split

In [ ]:
X = data._get_numeric_data().drop(['popularity'], axis=1)
y = data['popularity'].values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
X_train

,acousticness,danceability,liveness,loudness,speechiness,tempo,valence
65128,0.322288,0.275347,0.077378,0.823767,0.017501,0.705990,0.2430
117807,0.350401,0.576390,0.145739,0.793161,0.189648,0.646812,0.1970
41539,0.000090,0.568864,0.264892,0.831168,0.016653,0.475583,0.0398
113448,0.021184,0.593592,0.102319,0.815218,0.019941,0.475908,0.1720
90864,0.939759,0.568864,0.101310,0.719181,0.018350,0.505115,0.2770
...,...,...,...,...,...,...,...
110268,0.001113,0.351683,0.077176,0.835535,0.018456,0.522688,0.0993
119879,0.989960,0.123750,0.084548,0.611117,0.015062,0.225064,0.0367
103694,0.046184,0.657026,0.161896,0.859036,0.241621,0.267944,0.6400
131932,0.023192,0.552736,0.352741,0.866493,0.063110,0.331099,0.8140


### Linear Regression

In [ ]:
reg = LinearRegression().fit(X_train, y_train)

In [ ]:
y_pred_train = reg.predict(X_train)
print("Training accuracy: ", r2_score(y_train, y_pred_train))
scores = cross_val_score(reg, X_train, y_train, cv=10)
print("Mean cross-validation score: ", scores.mean())
y_pred_test = reg.predict(X_test)
print("Testing accuracy: ", r2_score(y_test, y_pred_test))

Training accuracy:  0.19633810910767635
Mean cross-validation score:  0.19611987690264526
Testing accuracy:  0.19672798091179933


### Ploynomial Regression

In [ ]:
poly = PolynomialFeatures(degree=2)
ploy_reg = LinearRegression().fit(poly.fit_transform(X_train), y_train)

In [ ]:
y_pred_train = ploy_reg.predict(poly.fit_transform(X_train))
print("Training accuracy: ", r2_score(y_train, y_pred_train))
scores = cross_val_score(ploy_reg, poly.fit_transform(X_train), y_train, cv=10)
print("Mean cross-validation score: ", scores.mean())
y_pred_test = ploy_reg.predict(poly.fit_transform(X_test))
print("Testing accuracy: ", r2_score(y_test, y_pred_test))

Training accuracy:  0.2337618150184103
Mean cross-validation score:  0.2331423900700679
Testing accuracy:  0.23285087382397673


In [ ]:
poly = PolynomialFeatures(degree=3)
ploy_reg = LinearRegression().fit(poly.fit_transform(X_train), y_train)

In [ ]:
y_pred_train = ploy_reg.predict(poly.fit_transform(X_train))
print("Training accuracy: ", r2_score(y_train, y_pred_train))
scores = cross_val_score(ploy_reg, poly.fit_transform(X_train), y_train, cv=10)
print("Mean cross-validation score: ", scores.mean())
y_pred_test = ploy_reg.predict(poly.fit_transform(X_test))
print("Testing accuracy: ", r2_score(y_test, y_pred_test))

Training accuracy:  0.2521181030952616
Mean cross-validation score:  0.25008781741643044
Testing accuracy:  0.25080712888381496


In [ ]:
poly = PolynomialFeatures(degree=4)
ploy_reg = LinearRegression().fit(poly.fit_transform(X_train), y_train)

In [ ]:
y_pred_train = ploy_reg.predict(poly.fit_transform(X_train))
print("Training accuracy: ", r2_score(y_train, y_pred_train))
scores = cross_val_score(ploy_reg, poly.fit_transform(X_train), y_train, cv=10)
print("Mean cross-validation score: ", scores.mean())
y_pred_test = ploy_reg.predict(poly.fit_transform(X_test))
print("Testing accuracy: ", r2_score(y_test, y_pred_test))

Training accuracy:  0.26850705127139374
Mean cross-validation score:  0.26206789253585333
Testing accuracy:  0.2642711243280901


In [ ]:
poly = PolynomialFeatures(degree=5)
ploy_reg = LinearRegression().fit(poly.fit_transform(X_train), y_train)

In [ ]:
y_pred_train = ploy_reg.predict(poly.fit_transform(X_train))
print("Training accuracy: ", r2_score(y_train, y_pred_train))
scores = cross_val_score(ploy_reg, poly.fit_transform(X_train), y_train, cv=10)
print("Mean cross-validation score: ", scores.mean())
y_pred_test = ploy_reg.predict(poly.fit_transform(X_test))
print("Testing accuracy: ", r2_score(y_test, y_pred_test))

Training accuracy:  0.28072723824896295
Mean cross-validation score:  0.2629617807045931
Testing accuracy:  0.26342044459373304


In [ ]:
poly = PolynomialFeatures(degree=6)
ploy_reg = LinearRegression().fit(poly.fit_transform(X_train), y_train)

In [ ]:
y_pred_train = ploy_reg.predict(poly.fit_transform(X_train))
print("Training accuracy: ", r2_score(y_train, y_pred_train))
scores = cross_val_score(ploy_reg, poly.fit_transform(X_train), y_train, cv=10)
print("Mean cross-validation score: ", scores.mean())
y_pred_test = ploy_reg.predict(poly.fit_transform(X_test))
print("Testing accuracy: ", r2_score(y_test, y_pred_test))

Training accuracy:  0.2936744529279671
Mean cross-validation score:  0.2368761797076105
Testing accuracy:  0.23912395907199058


In [ ]:
poly = PolynomialFeatures(degree=7)
ploy_reg = LinearRegression().fit(poly.fit_transform(X_train), y_train)
y_pred_train = ploy_reg.predict(poly.fit_transform(X_train))
print("Training accuracy: ", r2_score(y_train, y_pred_train))
scores = cross_val_score(ploy_reg, poly.fit_transform(X_train), y_train, cv=10)
print("Mean cross-validation score: ", scores.mean())
y_pred_test = ploy_reg.predict(poly.fit_transform(X_test))
print("Testing accuracy: ", r2_score(y_test, y_pred_test))

Training accuracy:  0.30954955246188876
Mean cross-validation score:  -0.49952444753106506
Testing accuracy:  -0.6184303254380246


### Support Vector Regression

In [ ]:
# svr = SVR().fit(X_train, y_train)
# score = svr.score(X_train, y_train)
# print("Training accuracy: ", score)
# scores = cross_val_score(svr, X_train, y_train,cv=10)
# print("Mean cross-validation score: ", scores.mean())
# y_pred_test = svr.predict(X_test)
# print("Testing accuracy: ", r2_score(y_test, y_pred_test))

### XGBoost Regression

In [ ]:
xgbr = xgb.XGBRegressor(verbosity = 0).fit(X_train, y_train)

In [ ]:
score = xgbr.score(X_train, y_train)
print("Training accuracy: ", score)
scores = cross_val_score(xgbr, X_train, y_train,cv=10)
print("Mean cross-validation score: ", scores.mean())
y_pred_test = xgbr.predict(X_test)
print("Testing accuracy: ", r2_score(y_test, y_pred_test))

Training accuracy:  0.2679272204214076
Mean cross-validation score:  0.2588841089257866
Testing accuracy:  0.2589175924026582


In [ ]:
parameters = {
    # Parameters that we are going to tune.
    'max_depth': [1,3,5,7,9,11,13],
    'min_child_weight': [1,2,3],
    'eta':[0.1, 0.3, 0.5],
    'subsample': [0.5, 1],
    'booster':["gbtree", "gblinear", "dart"],
}

In [ ]:
tuning_model = GridSearchCV(xgb.XGBRegressor(verbosity = 0), param_grid=parameters, scoring='neg_mean_squared_error', cv=3, verbose=3)

In [ ]:
# function for calculating how much time take for hyperparameter tuning
def timer(start_time=None):
    if not start_time:
        start_time=datetime.now()
        return start_time
    elif start_time:
        thour,temp_sec=divmod((datetime.now()-start_time).total_seconds(),3600)
        tmin,tsec=divmod(temp_sec,60)
        #print(thour,":",tmin,':',round(tsec,2))

In [ ]:
%%capture
from datetime import datetime

start_time=timer(None)

tuning_model.fit(X_train, y_train)

timer(start_time)

KeyboardInterrupt: ignored

In [ ]:
# best hyperparameters 
tuning_model.best_params_

NameError: ignored

### Decision Tree Regressor

In [ ]:
dt = DecisionTreeRegressor().fit(X_train, y_train)

In [ ]:
score = dt.score(X_train, y_train)
print("Training accuracy: ", score)
scores = cross_val_score(dt, X_train, y_train,cv=10)
print("Mean cross-validation score: ", scores.mean())
y_pred_test = dt.predict(X_test)
print("Testing accuracy: ", r2_score(y_test, y_pred_test))

Training accuracy:  0.9985309480510377
Mean cross-validation score:  -0.48709274621520465
Testing accuracy:  -0.4939555775420137


In [ ]:
# Hyper parameters range intialization for tuning 
parameters = {"splitter":["best","random"],
            "max_depth" : [1,3,5,7,9,11,13],
           "min_samples_leaf":[1,2,3,4,5,6,7,8,9,10],
           "min_weight_fraction_leaf":[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
           "max_features":["auto","log2","sqrt",None],
           "max_leaf_nodes":[None,10,20,30,40,50,60,70,80,90] }

In [ ]:
tuning_model = GridSearchCV(DecisionTreeRegressor(), param_grid=parameters, scoring='neg_mean_squared_error', cv=3, verbose=3)

In [ ]:
%%capture
from datetime import datetime

start_time=timer(None)

tuning_model.fit(X_train, y_train)

timer(start_time)

In [ ]:
# best hyperparameters 
tuning_model.best_params_

{'max_depth': 7,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_samples_leaf': 1,
 'min_weight_fraction_leaf': 0.1,
 'splitter': 'best'}

In [ ]:
tuned_hyper_model = DecisionTreeRegressor(max_depth=7 ,max_features='auto',max_leaf_nodes=None ,min_samples_leaf=1, min_weight_fraction_leaf=0.1, splitter='best')
dt = tuned_hyper_model.fit(X_train, y_train)

In [ ]:
score = dt.score(X_train, y_train)
print("Training accuracy: ", score)
scores = cross_val_score(dt, X_train, y_train,cv=10)
print("Mean cross-validation score: ", scores.mean())
y_pred_test = dt.predict(X_test)
print("Testing accuracy: ", r2_score(y_test, y_pred_test))

Training accuracy:  0.1851256575017689
Mean cross-validation score:  0.182544806204901
Testing accuracy:  0.18318239422175575


### Random Forest Regressor

In [ ]:
# this takes about 14min to run
rf = RandomForestRegressor().fit(X_train, y_train)
score = rf.score(X_train, y_train)
print("Training accuracy: ", score)
scores = cross_val_score(rf, X_train, y_train,cv=10)
print("Mean cross-validation score: ", scores.mean())
y_pred_test = rf.predict(X_test)
print("Testing accuracy: ", r2_score(y_test, y_pred_test))

Training accuracy:  0.8964059554418496
Mean cross-validation score:  0.2641509490295196
Testing accuracy:  0.2625501132516125


# Model Comparison

Requirments: 
* High cross validation and testing accuracy
* Reasonalble high training accuracy
* Under 5 second fitting time

Model(Tuned)|Training Accuracy|Cross Validation Accuracy(cv=10)|Testing Accuracy|Fitting Time    
----------------------------------|-----|-----|-----|-----|
Linear Regression                 |0.196|0.196|0.197|<1s 
Poly Regression (d=2)             |0.234|0.233|0.233|<1s
Poly Regression (d=3)             |0.252|0.250|0.251|1s
Poly Regression (d=4)             |0.269|0.262|0.264|3s
Poly Regression (d=5)             |0.281|0.263|0.263|11s
Poly Regression (d=6)             |0.294|0.237|0.240|38s
Support Vector Regressor          |na   |na   |na   |>1hour
XGBoost Regressor                 |0.268|0.259|0.259|11s
Desicion Tree Regressor    |0.185|0.183|0.183|<1s 
Random Forest Regressor   |0.896|0.264|0.263|81s


In [ ]:
Training accuracy:  0.2679272204214076
Mean cross-validation score:  0.2588841089257866
Testing accuracy:  0.2589175924026582

# Final model: Polynomial Regression(degree = 4)


In [ ]:
# Final model
X = data._get_numeric_data().drop(['popularity'], axis=1)
y = data['popularity'].values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

poly = PolynomialFeatures(degree=4)
ploy_reg = LinearRegression().fit(poly.fit_transform(X_train), y_train)
y_pred_train = ploy_reg.predict(poly.fit_transform(X_train))
print("Training accuracy: ", r2_score(y_train, y_pred_train))
scores = cross_val_score(ploy_reg, poly.fit_transform(X_train), y_train, cv=10)
print("Mean cross-validation score: ", scores.mean())
y_pred_test = ploy_reg.predict(poly.fit_transform(X_test))
print("Testing accuracy: ", r2_score(y_test, y_pred_test))